### Calculating PRS using VCF files


In [60]:
# making the notebok wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [56]:


# We can give a name to our app (to find it in Spark WebUI) and configure execution mode

APP_NAME="MAVANvcfPRS"
spark=SparkSession.builder.appName(APP_NAME).config("spark.ui.port", "4040").getOrCreate()
#conf = pyspark.SparkConf().setAppName(APP_NAME).set()#.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
sc = spark.sparkContext

sc.setLogLevel("WARN")
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
LOGGER.info("Start Reading Files")
#def main(gwasFile, genoFileList, thresholdList):

print(spark)
print(sc)

In [88]:
from __future__ import division
from pyspark import SparkConf, SparkContext
from operator import add
import re
import glob, os
import csv
from collections import Counter
import ntpath
import functools
#from functools import reduce
from math import log
import itertools
import PRS_VCF_utils
from time import time

import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.types import *

#**ATTN: python index starts at 0, so if you want to specify the second column, use 1
#**ATTN: please remove the header of the GWAS file if there is any

# define column number for contents in GWAS

gwas_id=0    # column of SNP ID
gwas_p=8     # column of P value
gwas_or=5    # column of odds ratio
gwas_a1=3    # column of a1 in the GWAS
gwas_maf= 4 # column index of maf in the GWAS

# defin column number for contents in genfile
geno_id=2  # column number with rsID
geno_start=9 # column number of the 1st genotype, in the raw vcf files, after separated by the delimiter of choice
geno_a1 = 3  # column number that contains the reference allele

# List of thresholds:
thresholds=[0.5]

# file delimiters:
GWAS_delim="\t"
GENO_delim="\t"

# file names:
home="/scratch/vvp-220-aa/KIDS_03info/"  #define homefolder path

#gwasFiles="/home/nyao111/PRS_imputed/pgc.mdd.clump.withAF.txt"       # Name of GWAS file 
gwasFiles="/home/nyao111/scratchbackup/GWAS-EWB-noheader-corrected-2.txt"


def getFileFromPattern(*pattern): # Multiple patterns need to be put into list format
    files=[]
    for pathpattern in pattern:
        files=glob.glob(files)

genoFileNamePattern=home+"*info03.vcf"   

genoFileNames=glob.glob(genoFileNamePattern)
# Alternatively, directly specify filename:
#genoFileName=[home+"fcgene_out_chr21comb.bierut1M_plus_filtered_chr21_c1_EA_COGA.gen",
              #home+"fcgene_out_chr21comb.bierut1M_plus_filtered_chr21_c1_EA_COGEND.gen",
              #home+"fcgene_out_chr22comb.bierut1M_plus_filtered_chr22_c1_EA_COGA.gen",
              #home+"fcgene_out_chr22comb.bierut1M_plus_filtered_chr22_c1_EA_COGEND.gen"]

genoExtension=".vcf"


# programme parameters
log_or=False  # sepcify whether you want to log your odds ratios
check_ref=True # if you know that there are mismatch between the top strand in the genotypes and that of the GWAS, set True. Not checking the reference allele will improve the speed

# sample file path and name
sampleFilePath=home+"MAVAN_35_impute161121_KIDS_orderedSamples.csv" # include the full/relative path and name of the sample file
sampleFileDelim=","  # sample File Delimiter
sampleFileID=0   # which column in the sample file has the ID
sample_skip=1  # how many lines to skip so that the sample names can be matched to the genotypes 1-to-1, taking into account the header of the sample file 
##output file information

outputPath=home+"MAVAN_KIDS_EWBPRS_161129.csv"



In [19]:
print("="*40)
print("Using these genoytpe files: ")

counter = 0
for filename in genoFileNames:
    if counter<20:
        counter+=1
        print(filename)
    else:
        print("and more...")
        print("total of {} files".format(str(len(genoFileNames))))
        break

Using these genoytpe files: 
/scratch/vvp-220-aa/KIDS_03info/KIDS10_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS21_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS22_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS6_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS4_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS8_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS17_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS12_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS11_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS19_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS14_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS5_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS7_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS15_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS16_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS20_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS3_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS2_info03.vcf
/scratch/vvp-220-aa/KIDS_03info/KIDS9_info03.vcf
/scratch/vvp-220-aa/KIDS_03in

### 1. Load files 

In [36]:
genodata=sc.textFile(genoFileNamePattern)
gwasfile=sc.textFile(gwasFiles)
print("Using the GWAS file: {}".format(ntpath.basename(gwasFiles)))
#gwastable=gwasfile.filter(lambda line: "snpid" not in line).map(lambda line: line.split(GWAS_delim))
gwastable=spark.read.option("header", "true").option("delimiter",GWAS_delim).csv(gwasFiles)
gwastable.show()
gwastableCA=gwastable.cache()


Using the GWAS file: GWAS-EWB-noheader-corrected-2.txt
+-----------+---+-----------+---+-----+-------+------+------+--------+------------+
|      SNPID|CHR|         BP| EA|  EAF|   Beta|    SE|    R2|p-value |N,Quasi-Repl|
+-----------+---+-----------+---+-----+-------+------+------+--------+------------+
|  rs3756290|  5|130,951,750|  A| 0.24|-0.0177|0.0031|0.011%|9.55E-09|     286,851|
|  rs2075677| 20| 47,701,024|  A| 0.76| 0.0175|0.0031|0.011%|1.49E-08|     288,454|
|  rs4958581|  5|152,187,729|  T| 0.66| 0.0153|0.0027|0.011%|2.29E-08|     294,043|
|  rs2572431|  8| 11,105,077|  T| 0.59| 0.0283|0.0035|0.039%|4.20E-16|     170,908|
|rs193236081| 17| 44,142,332|  T| 0.77|-0.0284|0.0043|0.028%|6.26E-11|     151,297|
| rs10960103|  9| 11,699,270|  C| 0.77| 0.0264|0.0042|0.024%|2.14E-10|     165,380|
|  rs4938021| 11|113,364,803|  T| 0.66| 0.0233|0.0037|0.024%|4.03E-10|     159,900|
|rs139237746| 11| 10,253,183|  T| 0.51|-0.0204|0.0034|0.021%|2.55E-09|     170,908|
|  rs1557341| 18| 35,

### 1.1 Filter GWAS and prepare odds ratio


In [37]:
reload(PRS_VCF_utils)
maxThreshold=max(thresholds)
gwasOddsMapMax=PRS_VCF_utils.filterGWASByP_DF(GWASdf=gwastableCA, pcolumn=gwas_p, idcolumn=gwas_id, oddscolumn=gwas_or, pHigh=maxThreshold, logOdds=log_or)
gwasOddsMapMaxCA=sc.broadcast(gwasOddsMapMax).value

Using the original values of effect sizes


### 2. Initial processing 

In [39]:
# at this step, the genotypes are already filtered to keep only the ones in 'gwasOddsMap'
genointermediate=genodata.filter(lambda line: ("#" not in line))\
.map(lambda line: line.split(GENO_delim))\
.filter(lambda line: line[geno_id] in gwasOddsMapMaxCA)\
.map(lambda line: line[0:5]+[chunk.split(":")[3] for chunk in line[geno_start::]])\
.map(lambda line: line[0:5]+[triplet.split(",") for triplet in line[5::]])

genoAlleles=genointermediate.map(lambda line: (line[geno_id], (line[geno_a1], line[geno_a1+1])))
genotable=genointermediate.map(lambda line: (line[geno_id], list(itertools.chain.from_iterable(line[5::]))))\
.mapValues(lambda geno: [float(x) for x in geno])


### 2.1 Calculate and store MAF

In [40]:
reload(PRS_VCF_utils)
genoa1f=genointermediate.map(lambda line: (line[geno_id], (line[geno_a1], line[geno_a1+1]), [float(x) for x in list(itertools.chain.from_iterable(line[5::]))]))\
.map(lambda line: (line[0], line[1][0], line[1][1], PRS_VCF_utils.getMaf(line[2]))).toDF(["Snpid_geno", "GenoA1", "GenoA2", "GenoA1f"])

#genoa1f.map(lambda line:"\t".join([line[0], "\t".join(line[1]), str(line[2])])).saveAsTextFile("../MOMS_info03_maf")


### 3. Determine whether each SNP needs to be flipped

In [57]:
gwasA1f=gwastableCA.rdd.map(lambda line:(line[gwas_id], line[gwas_a1], line[gwas_a1+1], line[gwas_maf])).toDF(["Snpid_gwas", "GwasA1", "GwasA2", "GwasMaf"])
checktable=genoa1f.join(gwasA1f, genoa1f["Snpid_geno"]==gwasA1f["Snpid_gwas"], "inner").cache()

In [64]:
# may take a long time
checktable.show()

+----------+------+------+-------------------+----------+------+------+-------+
|Snpid_geno|GenoA1|GenoA2|            GenoA1f|Snpid_gwas|GwasA1|GwasA2|GwasMaf|
+----------+------+------+-------------------+----------+------+------+-------+
|rs12903563|     T|     C| 0.5111111111111112|rs12903563|     T|  0.50|   0.50|
| rs1557341|     A|     C| 0.3672413793103448| rs1557341|     A|  0.34|   0.34|
| rs4958581|     T|     C| 0.6749042145593868| rs4958581|     T|  0.66|   0.66|
| rs2075677|     A|     G| 0.7739463601532567| rs2075677|     A|  0.76|   0.76|
|rs12961969|     A|     C|0.18735632183908044|rs12961969|     A|  0.20|   0.20|
|rs10838738|     A|     G| 0.6186781609195402|rs10838738|     A|  0.49|   0.49|
| rs3756290|     A|     G| 0.2674329501915708| rs3756290|     A|  0.24|   0.24|
| rs4481363|     C|     A|0.49885057471264377| rs4481363|     A| 0.524|  0.524|
| rs4481363|     C|     A|0.49885057471264377| rs4481363|     A|  0.49|   0.49|
| rs6904596|     G|     A| 0.90900383141

In [45]:
bpMap={"A":"T", "T":"A", "C":"G", "G":"C"}

In [77]:
reload(PRS_VCF_utils)
flagMap=checktable.rdd.map(lambda line: PRS_VCF_utils.checkAlignmentDF(line, bpMap)).collectAsMap()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


In [82]:
# filter the raw genotype file
reload(PRS_VCF_utils)
if check_ref:
    print("Calculating genotype dosage while taking into account strand alignment differences")
    genotypeMax=genotable.map(lambda line: PRS_VCF_utils.makeGenotypeCheckRef(line, checkMap=flagMap)).cache()
    samplesize=len(genotypeMax.first()[1])
else:
    genotypeMax=genotable.map(lambda line: PRS_VCF_utils.makeGenotype(line, gwasOddsMapCA)).cache()
    samplesize=len(genotypeMax.first()[1])

Calculating genotype dosage while taking into account strand alignment differences


In [83]:
# Calculate the PRS with the maximum threshold
# calculate PRS from genotype
reload(PRS_VCF_utils)
def calcPRSFromGeno(genotypeRDD, oddsMap):
    totalcount=genotypeRDD.count()
    multiplied=genotypeRDD.map(lambda line:[call * oddsMap[line[0]] for call in line[1]])
    filtered=multiplied.filter(lambda line: line is not None)
    PRS=multiplied.reduce(lambda a,b: map(add, a, b))
    normalizedPRS=[x/totalcount for x in PRS]
    return (totalcount,PRS)
start=time()
prsMax=calcPRSFromGeno(genotypeMax, gwasOddsMapMaxCA)
prsDict={}
prsDict[maxThreshold]=prsMax
print("finished calculating PRS at threshold of {}, used {:3.1f} seconds".format(str(maxThreshold), time()-start))
# Calculate PRS for the rest of the thresholds

finished calculating PRS at threshold of 0.5, used 381.9 seconds


In [34]:
def calcNoMax(genotypeRDD, gwasRDD, thresholdlist, prsMap):
    if len(thresholdlist)>1:
        thresholdListNoMax=[x for x in thresholds if x != maxThreshold]
        thresholdNoMaxSorted=sorted(thresholdListNoMax, reverse=True)
    else:
        thresholdNoMaxSorted=thresholdlist
    start=time()
    for threshold in thresholdNoMaxSorted:
        tic=time()
        gwasFiltered=PRS_VCF_utils.filterGWASByP(GWASRdd=gwasRDD, pcolumn=gwas_p, idcolumn=gwas_id, oddscolumn=gwas_or, pHigh=threshold, logOdds=log_or)
        print("filtering GWAS at threshold of {} took {:3.2f} seconds".format( str(threshold), time()-tic) )
        
        checkpoint=time()

        gwasFilteredBC=sc.broadcast(gwasFiltered)
        filteredgenotype=genotypeRDD.filter(lambda line: line[0] in gwasFilteredBC.value)
        if not filteredgenotype.isEmpty():
            prsOther=calcPRSFromGeno(filteredgenotype, gwasFilteredBC.value)
            prsMap[threshold]=prsOther
            print("finished calculating PRS at threshold of {}, used {:3.1f} seconds".format(str(threshold), time()-checkpoint))
            
    return prsMap

prsDict=calcNoMax(genotypeMax,gwastableCA, thresholds, prsDict)

Taking the log of odds ratios
filtering GWAS at threshold of 0.3 took 0.30 seconds
finished calculating PRS at threshold of 0.3, used 1.5 seconds
Taking the log of odds ratios
filtering GWAS at threshold of 0.2 took 0.27 seconds
finished calculating PRS at threshold of 0.2, used 1.3 seconds
Taking the log of odds ratios
filtering GWAS at threshold of 0.1 took 0.26 seconds
finished calculating PRS at threshold of 0.1, used 1.0 seconds
Taking the log of odds ratios
filtering GWAS at threshold of 0.05 took 0.22 seconds
finished calculating PRS at threshold of 0.05, used 0.9 seconds
Taking the log of odds ratios
filtering GWAS at threshold of 0.01 took 0.20 seconds
finished calculating PRS at threshold of 0.01, used 0.6 seconds
Taking the log of odds ratios
filtering GWAS at threshold of 0.001 took 0.18 seconds
finished calculating PRS at threshold of 0.001, used 0.7 seconds
Taking the log of odds ratios
filtering GWAS at threshold of 0.0001 took 0.18 seconds


In [89]:
reload(PRS_VCF_utils)
subjNames=PRS_VCF_utils.getSampleNames(sampleFilePath,sampleFileDelim,sampleFileID, skip=1)

In [93]:
import os
os.path.exists(outputPath)
os.path.exists("/home/nyao111")

True

In [94]:

reload(PRS_VCF_utils)
test=PRS_VCF_utils.writePRS(prsDict,  outputPath, samplenames=subjNames)

Successfully wrote scores to MAVAN_KIDS_EWBPRS_161129.csv
